In [5]:
fns = !ls fgs/sub-*.nii.gz
fns

['fgs/sub-122017_hemi-L_200um.nii.gz',
 'fgs/sub-122017_hemi-R_200um.nii.gz',
 'fgs/sub-152017_hemi-L_200um.nii.gz',
 'fgs/sub-152017_hemi-R_200um.nii.gz',
 'fgs/sub-bbhist_hemi-L_200um.nii.gz',
 'fgs/sub-bbhist_hemi-R_200um.nii.gz',
 'fgs/sub-pli3d_hemi-L_200um.nii.gz']

In [6]:
import os
import shutil
import subprocess
from glob import glob

# Create clean output directory
output_dir = 'layertemplate'
os.makedirs(output_dir, exist_ok=True)

# Clear existing contents
for f in glob(os.path.join(output_dir, '*')):
    os.remove(f)

# Open templatelist file for writing
csv_path = os.path.join(output_dir, 'templatelist.csv')
with open(csv_path, 'w') as csv_file:
    for i, fn in enumerate(fns):
        sub, hemi = os.path.basename(fn).split('_')[:2]

        out_fn = os.path.join(output_dir, f'tmp_{i}.nii.gz')

        if hemi[-1] == 'L':
            subprocess.run(['c3d', fn, '-flip', 'x', '-o', out_fn], check=True)
        else:
            shutil.copy(fn, out_fn)

        # Smooth each layer in the volume
        split_cmd = [
            'c3d', out_fn,
            '-split', '-foreach', '-smooth', '0.5x0.5x0.5mm', '-endfor',
            '-oo', os.path.join(output_dir, f'sub-{i}_label_%02d.nii.gz')
        ]
        subprocess.run(split_cmd, check=True)

        # Find smoothed label outputs
        label_paths = sorted(glob(os.path.join(output_dir, f'sub-{i}_label_*.nii.gz')))
        for lbl in label_paths:
            csv_file.write(f"{lbl},")
        csv_file.write("\n")
        for f in glob(os.path.join(output_dir, 'tmp_*.nii.gz')):
            os.remove(f)


KeyboardInterrupt: 

In [ ]:
!export ANTSPATH=/opt/minc/1.9.18/bin/
cmd = f"antsMultivariateTemplateConstruction2.sh -d 3 -o layertemplate/template_ layertemplate/templatelist.csv"
!{cmd}